In [1]:
# Import all the required libraries 

from glob import glob
import mne
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from scipy import stats
from scipy import signal
from scipy import fftpack
import pywt
import os,sys
from scipy.signal import coherence
from mne.time_frequency import psd_array_multitaper

In [2]:
# Import all of the EEG data

all_files = glob('F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\*.edf')
print(len(all_files))
print(all_files[0])

80
F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\H10_EC_post.edf


In [3]:
# Seperate date consisting of before eating patients and after eating patients

EC_pre = [i for i in all_files if 'EC_pre' in i.split("\\")[4]]
EC_post = [i for i in all_files if 'EC_post' in i.split("\\")[4]]

EC_pre.sort()
EC_post.sort()

print(EC_pre[19], EC_post[19])

F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\H9_EC_pre.edf F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\H9_EC_post.edf


In [4]:
def bandpower(data, sf, band, method='multitaper', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch' or 'multitaper'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)
    rel = bp / simps(psd, dx=freq_res)
    if relative:
        bp /= simps(psd, dx=freq_res)
    return [bp, rel]

In [5]:
%%capture
sf = 1000 # Sampling frequency

alpha = [8, 13]
beta = [13, 30]
delta = [1.25, 4]
theta = [4, 8]
gamma = [30, 48]

alpha_peak_pre_all = []

for file in EC_pre:
    raw = mne.io.read_raw_edf(file, preload=True, exclude=['EEG VREF'])
    raw = raw.crop(tmin=10, tmax=300)
    raw = (raw.copy()).filter(delta[0], gamma[1])

    alpha_patient = []

    for i in range(64):
        psd, freqs = psd_array_multitaper(raw.get_data()[i], sf, fmin=alpha[0], fmax=alpha[1], adaptive=True, normalization='full', verbose=0)
        index = np.argmax(psd)
        alpha_patient.append(freqs[index])

    alpha_peak_pre_all.append(alpha_patient)

In [6]:
%%capture
sf = 1000 # Sampling frequency

alpha = [8, 13]
beta = [13, 30]
delta = [1.25, 4]
theta = [4, 8]
gamma = [30, 48]

alpha_peak_post_all = []

for file in EC_post:
    raw = mne.io.read_raw_edf(file, preload=True, exclude=['EEG VREF'])
    raw = raw.crop(tmin=10, tmax=300)
    raw = (raw.copy()).filter(delta[0], gamma[1])

    alpha_patient = []

    for i in range(64):
        psd, freqs = psd_array_multitaper(raw.get_data()[i], sf, fmin=alpha[0], fmax=alpha[1], adaptive=True, normalization='full', verbose=0)
        index = np.argmax(psd)
        alpha_patient.append(freqs[index])

    alpha_peak_post_all.append(alpha_patient)

Looking into the relationship between all of the channels instead of combining all channels into one

In [7]:
cols = []
bands = ['alpha']

for i in range(1):
    for j in range(64):
        cols.append(bands[i] + str(j + 1))

cols.append('target')

In [8]:
all_combined_pre = []
all_combined_post = []

for i in range(20):
    all_combined_pre.append(alpha_peak_pre_all[i] + [1])
    all_combined_post.append(alpha_peak_post_all[i] + [0])

In [9]:

all_combined = all_combined_pre + all_combined_post
df = pd.DataFrame(all_combined, columns = cols, dtype = float)
df.to_excel('Dataset_alpha_peak.xlsx')

Computing pairwise T plots for all powers

In [10]:
alpha_peak_pre_all_t = np.array(alpha_peak_pre_all).T.tolist()
alpha_peak_post_all_t = np.array(alpha_peak_post_all).T.tolist()

def compute_ttest(pre, post):
    results = []
    for i in range(64):
        results.append(stats.ttest_rel(pre[i], post[i]))
        if(results[i].pvalue <= 0.05):
            print(i + 1)

print("ALPHA")
compute_ttest(alpha_peak_pre_all_t, alpha_peak_post_all_t)

ALPHA
1
9
21
26
48
49
